In [17]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

import requests

from bs4 import BeautifulSoup

import time

from selenium.webdriver.common.keys import Keys

import os
import re

## Q1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [10]:
try:
    driver = webdriver.Edge()
    driver.get('https://www.amazon.in/')
    # Enter “guitar” in the search field 
    product_name = input("Search for the product which you are interested in ")
    product_search = driver.find_element(By.XPATH, "//input[@class='nav-input nav-progressive-attribute']")  
    product_search.send_keys(product_name)
    time.sleep(3)
    
    button_product = driver.find_element(By.XPATH, "//div[@class='nav-search-submit nav-sprite']")  
    button_product.click()
except NoSuchElementException as e:
    print("Exception Raised ", e)
    print("Refreshing the Page ")
    time.sleep(3)
    driver.get('https://www.amazon.in/')
    
    product_name = input("Search for the product which you are interested in ")
    product_search = driver.find_element(By.XPATH, "//input[@class='nav-input nav-progressive-attribute']")  
    product_search.send_keys(product_name)
    time.sleep(3)
    
    button_product = driver.find_element(By.XPATH, "//div[@class='nav-search-submit nav-sprite']")  
    button_product.click()
    

Search for the product which you are interested in guitar


## Q2 In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. 
Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. 
In case, if any of the details are missing for any of the product then replace it by “-“. 

In [11]:
# Fetching product 3 pages URL

# Product URL

product_page_url = []

start = 0
end = 3

for page in range(start, end):
    url = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')  
    for i in url:
        product_page_url.append(i.get_attribute('href'))
    next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
    time.sleep(3)
    
print(len(product_page_url))

173


In [12]:
print(product_page_url)

['https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzE5Njg0NDQyNDIzNjM5OjE2OTYyNDcxNzM6c3BfYXRmOjMwMDAxNTUzNzk3OTIzMjo6MDo6&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB078GTJP5Y%2Fref%3Dsr_1_1_sspa%3Fcrid%3D275XQA48GYQN9%26keywords%3Dguitar%26qid%3D1696247173%26sprefix%3Dguitar%252Caps%252C207%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1', 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzE5Njg0NDQyNDIzNjM5OjE2OTYyNDcxNzM6c3BfYXRmOjIwMDk2NjA4NzUwNDk4OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fref%3Dsr_1_2_sspa%3Fcrid%3D275XQA48GYQN9%26keywords%3Dguitar%26qid%3D1696247173%26sprefix%3Dguitar%252Caps%252C207%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1', 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzE5Njg0NDQyNDIzNjM5OjE2OTYyNDcxNzM6c3BfYXRmOjMwMDA1MTU4ODUxNjAzMjo6MDo6&url=%2FRevel-RVL-38C-LGP-BLS-Linden-Acoustic-Guitar%2Fdp%2FB085HWKDCY%2Fref%3Dsr_1_3_sspa%3Fcrid%3D275XQA48GYQN9%26keywords%3Dguitar%26qi

In [13]:
brand_name = []
product_name = []
price = []
Return_Exchange = []
Expected_Delivery = []
availability = []


for url in product_page_url:
    
    driver.get(url)
    
    time.sleep(3)
    
    # 1. Brand Name
    
    brand_element = driver.find_elements(By.XPATH, "//span[@class='a-size-large product-title-word-break']")
    time.sleep(2)
        
    for i in brand_element:
        if(i.text is ''):
            brand_name.append('--')
        else:
            brand_name.append(i.text.split(' ')[0])
                             
    # 2. Product Name
    
    product_element = driver.find_elements(By.XPATH, "//span[@class='a-size-large product-title-word-break']")
    time.sleep(2)
        
    for i in product_element:
        if(i.text is ''):
            product_name.append('--')
        else:
            product_name.append(i.text)
    
    # 3. Price
    
    price_element = driver.find_elements(By.XPATH, "//span[@class='a-price aok-align-center']")
    time.sleep(2)
        
    for i in price_element:
        if(i.text is ''):
            price.append('--')
        else:
            price.append(i.text.split('\n')[0])
       
    # 4. Return/Exchange
    
    Return_Exchange_element = driver.find_elements(By.XPATH, "//span[@class='a-size-small a-color-link a-text-normal']")
    time.sleep(2)
        
    count = 0
    for i in Return_Exchange_element:
        if(count == 2):
            if i.text is '':
                Return_Exchange.append('--')
            else:
                Return_Exchange.append(i.text)
            break
        count = count + 1
        
    # 5. Expected Delivery
    
    Expected_Delivery_element = driver.find_elements(By.XPATH, "//*[@id='mir-layout-DELIVERY_BLOCK']")
    time.sleep(2)
        
    for i in Expected_Delivery_element:
        if(i.text is ''):
            Expected_Delivery.append('--')
        else:
            Expected_Delivery.append(i.text)
        
    # 6. Availablility
    
    availability_element = driver.find_elements(By.XPATH, '//div[@class="a-section a-spacing-none a-spacing-top-micro }"]')
    time.sleep(2)
        
    for i in availability_element:
        if(i.text is ''):
            availability.append('--')
        else:
            availability.append(i.text)       
    
    

In [14]:
print(len(brand_name), len(product_name), len(price), len(Return_Exchange), len(Expected_Delivery), len(availability))
print(availability)

173 173 170 171 170 170
['In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'Only 2 left in stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'Only 1 left in stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'Only 1 left in stock', 'In stock', 'In stock', 'In stock', 'Only 1 left in stock', 'In stock', 'In stock', 'In stock', 'Only 2 left in stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'Only 2 left in stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'Only 1 left in stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In stock', 'In

## Q2 Output

In [15]:
df = pd.DataFrame({
    "Brand Name": brand_name[:170],
    "Product Name": product_name[:170],
    "Price": price[:170],
    "Return/Exchange": Return_Exchange[:170],
    "Expected Delivery": Expected_Delivery[:170],
    "Availability": availability[:170],
    "Product URL": product_page_url[:170]
})
df

,Brand Name,Product Name,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","₹4,999",7 days Replacement,"FREE delivery Thursday, 5 October. Details\nOr...",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,"₹4,299",7 days Replacement,"FREE delivery Thursday, 5 October. Details\nOr...",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Revel,Revel Acoustic Guitar with Carry bag and Plect...,"₹1,999",7 days Replacement,"FREE delivery Sunday, 8 October. Details",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,“ALL,"“ALL WOODEN” 38” INCH ""DOLPHIN"" Guitar Combo P...","₹2,950",7 days Returnable,"FREE delivery Wednesday, 11 October. Order wit...",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹1,999",7 days Replacement,"FREE delivery Saturday, 7 October. Details\nOr...",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
165,INTERN,INTERN 41 inches Acoustic Guitar with truss ro...,"₹7,015",7 days Returnable,"FREE delivery Thursday, 5 October. Details\nOr...",In stock,https://www.amazon.in/Acoustic-plectrums-stabi...
166,D'Addario,"D'Addario EZ900 Acoustic Guitar Strings, Pack ...","₹3,799",7 days Replacement,"FREE delivery Wednesday, 4 October. Order with...",In stock,https://www.amazon.in/DAddario-EZ900-Acoustic-...
167,Kadence,Kadence 12 strings Slowhand Premium Mahogany w...,"₹3,419",7 days Replacement,"FREE delivery Saturday, 7 October. Order withi...",In stock,https://www.amazon.in/Kadence-Slowhand-Mahogan...
168,JUAREZ,"JUAREZ Arpeggio Guitar JRA39SR-BL-K, blue","₹11,919",7 Day Warranty,"FREE delivery Friday, 6 October. Details\nOr f...",In stock,https://www.amazon.in/JUAREZ-Arpeggio-Guitar-J...


In [16]:
df.to_csv('Guitar.csv', index=False)

## Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [18]:
# Function to scrape images for keywords ‘fruits’, ‘cars’, ‘Machine Learning’, ‘Guitar’, ‘Cakes’
def scrape_images(keyword):
    driver = webdriver.Edge()
    driver.get('https://images.google.com/')
    
    # Enter keywords in the search field 
    keyword_search = driver.find_element(By.XPATH, "//textarea[@class='gLFyf']")  
    keyword_search.send_keys(keyword)
    keyword_search.send_keys(Keys.RETURN)
    
    time.sleep(2)

    # scroll, bottom of the page to load more images
    for _ in range(30):
        driver.execute_script("window.scrollBy(0, 100)")
    
    time.sleep(1)

    image_elements = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')

    image_urls = []

    # Extract image URLs
    for i in image_elements:
        source = i.get_attribute('src')
        if(source is not None):
            if(source[0:4] == 'http'):
                image_urls.append(source)
                
    # to download images in our local drive
    os.makedirs(keyword, exist_ok=True)
    for i, j in enumerate(image_urls[:10]):
        img_data = requests.get(j).content
        with open(f"{keyword}/image_{i+1}.jpg", 'wb') as handler:
            handler.write(img_data)

        
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

# Iterate and scrape images for each in keywords
for i in keywords:
    scrape_images(i)
    print("Downloaded Images", i)

Downloaded Images fruits
Downloaded Images cars
Downloaded Images Machine Learning
Downloaded Images Guitar
Downloaded Images Cakes


## Q3 Output - all images ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’ are stored in folder


## Q4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV. 
    

In [ ]:
driver = webdriver.Edge()

In [330]:
# Go to Flipkart webpage by url :https://www.flipkart.com/

driver.get('https://www.flipkart.com/')

In [331]:
# Enter “Oneplus Nord” in the search field where “search for products, brands and more” is written 

Oneplus_Nord_search = driver.find_element(By.XPATH, "//input[@class='Pke_EE']")  
Oneplus_Nord_search.send_keys('Oneplus Nord')

In [332]:
# and click the search icon

button_Oneplus_Nord = driver.find_element(By.XPATH, "//button[@class='_2iLD__']")  
button_Oneplus_Nord.click()

In [333]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 1. Brand Name

brand = []
brand_element = driver.find_elements(By.XPATH, "//div[@class='_4rR01T']")
time.sleep(2)
        
for i in brand_element:
    brand.append(i.text.split(' ')[0])

In [334]:
print(len(brand))
print(brand)

24
['OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus', 'OnePlus']


In [335]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 2. Smartphone name

Smartphone_name = []
Smartphone_element = driver.find_elements(By.XPATH, "//div[@class='_4rR01T']")
time.sleep(2)

for i in Smartphone_element:
    Smartphone_name.append(i.text)

In [336]:
print(len(Smartphone_name))
print(Smartphone_name)

24
['OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)', 'OnePlus Nord CE 3 Lite 5G (Pastel Lime, 256 GB)', 'OnePlus Nord CE 3 Lite 5G (Pastel Lime, 128 GB)', 'OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)', 'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256 GB)', 'OnePlus Nord CE3 5G (Aqua Surge, 128 GB)', 'OnePlus Nord CE3 5G (Grey Shimmer, 256 GB)', 'OnePlus Nord CE3 5G (Aqua Surge, 256 GB)', 'OnePlus Nord 3 5G (Misty Green, 128 GB)', 'OnePlus Nord 3 5G (Misty Green, 256 GB)', 'OnePlus Nord 3 5G (Tempest Gray, 128 GB)', 'OnePlus Nord CE 2 5G (Gray Mirror, 128 GB)', 'OnePlus Nord 2T 5G (Gray Shadow, 128 GB)', 'OnePlus Nord CE3 5G (Grey Shimmer, 128 GB)', 'OnePlus Nord 3 5G (Tempest Gray, 256 GB)', 'OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)', 'OnePlus Nord 2T 5G (Jade Fog, 256 GB)', 'OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)', 'OnePlus Nord CE 5G (Charcoal Ink, 128 GB)', 'OnePlus Nord 2 5G (Gray Sierra, 128 GB)', 'OnePlus Nord 2 5G (Gray Sierra, 256 GB)', 'OnePlus Nord 2 5G (PAC-MAN, 256 

In [337]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 3. Colour

import re

Colour = []


Colour_element = driver.find_elements(By.XPATH, "//div[@class='_4rR01T']")
time.sleep(2)

for i in Colour_element:
    r = re.findall(r"[A-Za-z]+\s[A-Za-z]+", i.text)
    r.pop(0)
    if (len(r) == 0):
        r.append('-')
            
    Colour.append(r)

In [338]:
print(len(Colour))
print(Colour)

24
[['Blue Tide'], ['Pastel Lime'], ['Pastel Lime'], ['Black Dusk'], ['Chromatic Gray'], ['Aqua Surge'], ['Grey Shimmer'], ['Aqua Surge'], ['Misty Green'], ['Misty Green'], ['Tempest Gray'], ['Gray Mirror'], ['Gray Shadow'], ['Grey Shimmer'], ['Tempest Gray'], ['Bahama Blue'], ['Jade Fog'], ['Bahama Blue'], ['Charcoal Ink'], ['Gray Sierra'], ['Gray Sierra'], ['-'], ['Gray Sierra'], ['Chromatic Gray']]


In [339]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# RAM and Storage(ROM)

try:
    RAM_ROM = []
    RAM_ROM_element = driver.find_elements(By.XPATH, "//ul[@class='_1xgFaf']")
    time.sleep(2)

    for i in RAM_ROM_element:
        RAM_ROM.append(i.text.split('\n')[0])
except:
    RAM_ROM.append('-')
    

In [340]:
print(len(RAM_ROM))
print(RAM_ROM)

24
['6 GB RAM | 128 GB ROM', '8 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '6 GB RAM | 128 GB ROM', '8 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '12 GB RAM | 256 GB ROM', '12 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '16 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '6 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '16 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '12 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '12 GB RAM | 256 GB ROM', '12 GB RAM | 256 GB ROM', '12 GB RAM | 256 GB ROM', '8 GB RAM | 128 GB ROM']


In [341]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 4. RAM

RAM = []

try:
    for i in RAM_ROM:
        RAM.append(i.split('|')[0])
except IndexError:
    RAM.append('-')

In [342]:
print(len(RAM))
print(RAM)

24
['6 GB RAM ', '8 GB RAM ', '8 GB RAM ', '6 GB RAM ', '8 GB RAM ', '8 GB RAM ', '12 GB RAM ', '12 GB RAM ', '8 GB RAM ', '16 GB RAM ', '8 GB RAM ', '6 GB RAM ', '8 GB RAM ', '8 GB RAM ', '16 GB RAM ', '8 GB RAM ', '12 GB RAM ', '8 GB RAM ', '8 GB RAM ', '8 GB RAM ', '12 GB RAM ', '12 GB RAM ', '12 GB RAM ', '8 GB RAM ']


In [343]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 5. Storage(ROM)

ROM = []

try:
    for i in RAM_ROM:
        ROM.append(i.split('|')[1])
except:
    ROM.append('-')

In [344]:
print(len(ROM))
print(ROM)

24
[' 128 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 128 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 256 GB ROM', ' 128 GB ROM']


In [345]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# Primary Camera and Secondary Camera

try:
    Camera = []
    Camera_element = driver.find_elements(By.XPATH, "//ul[@class='_1xgFaf']")
    time.sleep(2)

    for i in Camera_element:
        Camera.append(i.text.split('\n')[2])
except:
    Camera.append('-')

In [346]:
print(len(Camera))
print(Camera)

24
['64MP Rear Camera | 16MP Front Camera', '108MP Rear Camera', '108MP Rear Camera', '64MP Rear Camera | 16MP Front Camera', '108MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera | 32MP Front Camera', '50MP Rear Camera', '50MP Rear Camera', '64MP Rear Camera', '50MP Rear Camera', '64MP Rear Camera', '64MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '108MP Rear Camera']


In [347]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 6. Primary Camera

Primary_Camera = []

try:
    for i in Camera:
        Primary_Camera.append(i.split('|')[0])
except:
    Primary_Camera.append('-')

In [348]:
print(len(Primary_Camera))
print(Primary_Camera)

24
['64MP Rear Camera ', '108MP Rear Camera', '108MP Rear Camera', '64MP Rear Camera ', '108MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera ', '50MP Rear Camera', '50MP Rear Camera', '64MP Rear Camera', '50MP Rear Camera', '64MP Rear Camera', '64MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '50MP Rear Camera', '108MP Rear Camera']


In [349]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 7. Secondary Camera

Secondary_Camera = []
for i in Camera:
    try:
        Secondary_Camera.append(i.split('|')[1])
    except IndexError as e:
        Secondary_Camera.append('-')
    continue

In [350]:
print(len(Secondary_Camera))
print(Secondary_Camera)

24
[' 16MP Front Camera', '-', '-', ' 16MP Front Camera', '-', '-', '-', '-', '-', '-', '-', '-', ' 32MP Front Camera', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


In [351]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 8. Display Size

try:
    Display_Size = []
    Display_Size_element = driver.find_elements(By.XPATH, "//ul[@class='_1xgFaf']")
    time.sleep(2)

    for i in Display_Size_element:
        Display_Size.append(i.text.split('\n')[1])
except:
    Display_Size.append('-')

In [352]:
print(len(Display_Size))
print(Display_Size)

24
['16.74 cm (6.59 inch) Display', '17.07 cm (6.72 inch) Display', '17.07 cm (6.72 inch) Display', '16.74 cm (6.59 inch) Display', '17.07 cm (6.72 inch) Display', '17.02 cm (6.7 inch) Display', '17.02 cm (6.7 inch) Display', '17.02 cm (6.7 inch) Display', '17.12 cm (6.74 inch) Display', '17.12 cm (6.74 inch) Display', '17.12 cm (6.74 inch) Display', '16.33 cm (6.43 inch) Display', '17.02 cm (6.7 inch) Display', '17.02 cm (6.7 inch) Display', '17.12 cm (6.74 inch) Display', '16.33 cm (6.43 inch) Display', '17.02 cm (6.7 inch) Display', '16.33 cm (6.43 inch) Display', '16.33 cm (6.43 inch) Display', '16.33 cm (6.43 inch) Display', '16.33 cm (6.43 inch) Display', '16.33 cm (6.43 inch) Display', '16.33 cm (6.43 inch) Display', '17.07 cm (6.72 inch) Display']


In [353]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 9. Battery Capacity

try:
    Battery_Capacity = []
    Battery_Capacity_element = driver.find_elements(By.XPATH, "//ul[@class='_1xgFaf']")
    time.sleep(2)

    for i in Battery_Capacity_element:
        Battery_Capacity.append(i.text.split('\n')[3])
except:
    Battery_Capacity.append('-')

In [354]:
print(len(Battery_Capacity))
print(Battery_Capacity)

24
['5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '4500 mAh Battery', '5000 mAh Battery']


In [355]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 10. Price

try:
    Price = []
    Price_element = driver.find_elements(By.XPATH, "//div[@class='_30jeq3 _1_WHN1']")
    time.sleep(2)

    for i in Price_element:
        Price.append(i.text)
except:
    Price.append('-')

In [356]:
print(Price)
print(len(Price))

['₹17,210', '₹21,997', '₹19,999', '₹17,101', '₹21,999', '₹26,129', '₹28,840', '₹28,978', '₹32,449', '₹37,945', '₹31,752', '₹23,990', '₹28,948', '₹26,649', '₹37,970', '₹24,990', '₹32,358', '₹24,799', '₹24,990', '₹30,599', '₹34,999', '₹37,999', '₹33,990', '₹19,989']
24


In [357]:
# From this page you can scrap the required data as usual.
# You have to scrape “Brand name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”

# 11. Product URL

product_url = []

url = driver.find_elements(By.XPATH, '//a[@class="_1fQZEK"]')  
for i in url:
    product_url.append(i.get_attribute('href'))

In [358]:
print(product_url)
print(len(product_url))

['https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGMFREBAHZQGY9&lid=LSTMOBGMFREBAHZQGY9HACYLB&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&spotlightTagId=BestsellerId_tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=a8d9364d-97de-4c51-a699-2b6e9b6d3fa4.MOBGMFREBAHZQGY9.SEARCH&ppt=sp&ppn=sp&ssid=j1xzmmexgg0000001696002236078&qH=53863d4d978d5074', 'https://www.flipkart.com/oneplus-nord-ce-3-lite-5g-pastel-lime-256-gb/p/itm2cd5a4e659035?pid=MOBGZJ3ZFQ8AKKRS&lid=LSTMOBGZJ3ZFQ8AKKRSDHXFLS&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=a8d9364d-97de-4c51-a699-2b6e9b6d3fa4.MOBGZJ3ZFQ8AKKRS.SEARCH&ppt=hp&ppn=homepage&ssid=j1xzmmexgg0000001696002236078&qH=53863d4d978d5074', 'https://www.flipkart.com/oneplus-nord-ce-3-lite-5g-pastel-lime-128-gb/p/itm2cd5a4e659035?pid=MOBGZJ3WM5SGTGVZ&lid=LSTMOBGZJ3WM5SGTGVZJA1PMS&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4

## Q4 Output

In [360]:
data = {
    'Brand Name': brand,
    'Smartphone Name': Smartphone_name,
    'Color': Colour,
    'RAM': RAM,
    'Storage(ROM)': ROM,
    'Primary Camera': Primary_Camera,
    'Secondary Camera': Secondary_Camera,
    'Display Size': Display_Size,
    'Battery Capacity': Battery_Capacity,
    'Price': Price,
    'Product URL': product_url
}

df = pd.DataFrame(data)
df


,Brand Name,Smartphone Name,Color,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",[Blue Tide],6 GB RAM,128 GB ROM,64MP Rear Camera,16MP Front Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹17,210",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"OnePlus Nord CE 3 Lite 5G (Pastel Lime, 256 GB)",[Pastel Lime],8 GB RAM,256 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch) Display,5000 mAh Battery,"₹21,997",https://www.flipkart.com/oneplus-nord-ce-3-lit...
2,OnePlus,"OnePlus Nord CE 3 Lite 5G (Pastel Lime, 128 GB)",[Pastel Lime],8 GB RAM,128 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch) Display,5000 mAh Battery,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-3-lit...
3,OnePlus,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",[Black Dusk],6 GB RAM,128 GB ROM,64MP Rear Camera,16MP Front Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹17,101",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256...",[Chromatic Gray],8 GB RAM,256 GB ROM,108MP Rear Camera,-,17.07 cm (6.72 inch) Display,5000 mAh Battery,"₹21,999",https://www.flipkart.com/oneplus-nord-ce-3-lit...
5,OnePlus,"OnePlus Nord CE3 5G (Aqua Surge, 128 GB)",[Aqua Surge],8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹26,129",https://www.flipkart.com/oneplus-nord-ce3-5g-a...
6,OnePlus,"OnePlus Nord CE3 5G (Grey Shimmer, 256 GB)",[Grey Shimmer],12 GB RAM,256 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹28,840",https://www.flipkart.com/oneplus-nord-ce3-5g-g...
7,OnePlus,"OnePlus Nord CE3 5G (Aqua Surge, 256 GB)",[Aqua Surge],12 GB RAM,256 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹28,978",https://www.flipkart.com/oneplus-nord-ce3-5g-a...
8,OnePlus,"OnePlus Nord 3 5G (Misty Green, 128 GB)",[Misty Green],8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.12 cm (6.74 inch) Display,5000 mAh Battery,"₹32,449",https://www.flipkart.com/oneplus-nord-3-5g-mis...
9,OnePlus,"OnePlus Nord 3 5G (Misty Green, 256 GB)",[Misty Green],16 GB RAM,256 GB ROM,50MP Rear Camera,-,17.12 cm (6.74 inch) Display,5000 mAh Battery,"₹37,945",https://www.flipkart.com/oneplus-nord-3-5g-mis...


In [ ]:
df.to_csv('Oneplus.csv', index=False)

## Q5 Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [20]:
# Function to get geospatial coordinates from Google Maps webpage
def geospatial_coordinates(city):
    
    driver = webdriver.Edge()
    
    # Go to google maps webpage https://www.google.co.in/maps

    driver.get('https://www.google.co.in/maps')
    time.sleep(2)

    # search for the city
    city_search = driver.find_element(By.XPATH, "//input[@class='searchboxinput xiQnY']")  
    city_search.send_keys(city)
    time.sleep(3)  

    # click the search icon
    city_button = driver.find_element(By.XPATH, "//button[@class='mL3xi google-symbols']")  
    city_button.click()
    time.sleep(2)
    
    print("URL Extracted ", driver.current_url)

    url = driver.current_url
   
    latitude_longitude = re.findall(r'@(.*)data',url)
    #print(latitude_longitude)
    
    if len(latitude_longitude):
        latitude_longitude_list = latitude_longitude[0].split(",")
        if len(latitude_longitude_list)>=2:
            latitude = latitude_longitude_list[0]
            longitude = latitude_longitude_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))


city = input("enter city name you want to search  ")
geospatial_coordinates(city)


enter city name you want to search  Bangalore
URL Extracted  https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@12.9544595,77.3012649,10z/data=!3m1!4b1!4m6!3m5!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627!16zL20vMDljMTc?entry=ttu
Latitude = 12.9544595, Longitude = 77.3012649


## Q5 Output - Extracted Latitude = 12.9544595, Longitude = 77.3012649 of Bangalore City

## Q6 Write a program to scrap all the available details of best gaming laptops from digit.in

In [248]:
driver = webdriver.Edge()

In [251]:
# Go to best gaming laptop webpage https://www.digit.in/top-products/best-amd-gaming-laptops-for-students-4071.html

driver.get('https://www.digit.in/top-products/best-amd-gaming-laptops-for-students-4071.html')

In [252]:
# 1. laptop Name

laptop_name = []
laptop_name_element = driver.find_elements(By.XPATH, "//h3[@class='font130 mt0 mb10 mobilesblockdisplay ']")
time.sleep(2)
        
for i in laptop_name_element:
    laptop_name.append(i.text)

In [253]:
print(laptop_name)
print(len(laptop_name))

['HP Victus 16-e0362ax 5th Gen Ryzen 7-5800H (2022)', 'Acer Nitro 5 AN515-45-R3TC Ryzen 5-5600H (2021)', 'Dell New G15-5515 D560804WIN9W Ryzen 5-5600H (2022)', 'HP Pavilion Gaming 15-EC2048AX Ryzen 5-5600X', 'Lenovo Ideapad Gaming 3 82K201RRIN AMD Ryzen 5-5600H (2022)']
5


In [259]:
# 2. laptop details

laptop_details = []
laptop_details_element = driver.find_elements(By.XPATH, "//div[@class='rh-flex-center-align']")
time.sleep(2)
        
for i in laptop_details_element:
    laptop_details.append(i.text.split('\n')[1])


In [260]:
print(laptop_details)
print(len(laptop_details))

['Windows 11 Home', '16.1', '2560 x 1440', '5th Gen AMD Ryzen 7-5800H', 'Windows 10 Home', '15.6', '1920 x 1080', 'AMD Ryzen 5 Hexa Core 5600H', 'Windows 11 Home', '15.6', '1920 x 1080', 'AMD Ryzen 5-5600H', 'Windows 10 Home', '15.6', '1920 x 1080', 'AMD Ryzen 5-5600X', 'Windows 11 Home', '15.6', '1920 x 1080', 'AMD Ryzen 5-5600H']
20


In [261]:
Operating_System = []
Display_Size = []
Resolution = []
Processor  = []

for i in range(0, len(laptop_details), 4):
    Operating_System.append(laptop_details[i])
print(Operating_System)

for i in range(1, len(laptop_details), 4):
    Display_Size.append(laptop_details[i])
print(Display_Size)

for i in range(2, len(laptop_details), 4):
    Resolution.append(laptop_details[i])
print(Resolution)

for i in range(3, len(laptop_details), 4):
    Processor.append(laptop_details[i])
print(Processor)

['Windows 11 Home', 'Windows 10 Home', 'Windows 11 Home', 'Windows 10 Home', 'Windows 11 Home']
['16.1', '15.6', '15.6', '15.6', '15.6']
['2560 x 1440', '1920 x 1080', '1920 x 1080', '1920 x 1080', '1920 x 1080']
['5th Gen AMD Ryzen 7-5800H', 'AMD Ryzen 5 Hexa Core 5600H', 'AMD Ryzen 5-5600H', 'AMD Ryzen 5-5600X', 'AMD Ryzen 5-5600H']


## Q6 Output

In [262]:
df = pd.DataFrame({
    'Laptop Name' : laptop_name,
    'Operating System':Operating_System,
    'Display Size':Display_Size,
    'Resolution':Resolution,
    'Processor':Processor
})
df

,Laptop Name,Operating System,Display Size,Resolution,Processor
0,HP Victus 16-e0362ax 5th Gen Ryzen 7-5800H (2022),Windows 11 Home,16.1,2560 x 1440,5th Gen AMD Ryzen 7-5800H
1,Acer Nitro 5 AN515-45-R3TC Ryzen 5-5600H (2021),Windows 10 Home,15.6,1920 x 1080,AMD Ryzen 5 Hexa Core 5600H
2,Dell New G15-5515 D560804WIN9W Ryzen 5-5600H (...,Windows 11 Home,15.6,1920 x 1080,AMD Ryzen 5-5600H
3,HP Pavilion Gaming 15-EC2048AX Ryzen 5-5600X,Windows 10 Home,15.6,1920 x 1080,AMD Ryzen 5-5600X
4,Lenovo Ideapad Gaming 3 82K201RRIN AMD Ryzen 5...,Windows 11 Home,15.6,1920 x 1080,AMD Ryzen 5-5600H


## Q7 Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [40]:
driver = webdriver.Edge()

In [41]:
# Go to webpage https://www.forbes.com/billionaires/

driver.get('https://www.forbes.com/billionaires/')

In [42]:
# To scarpe data (i.e. “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”) 

# All

all = []
all_element = driver.find_elements(By.XPATH, "//div[@class='TableRow_cell__db-hv Table_cell__houv9']")
time.sleep(2)
        
for i in all_element:
    all.append(i.text)

In [43]:
print(all)
print(len(all))

['1', 'Bernard Arnault & family', '$211 B', '74', 'France', 'LVMH', 'Fashion & Retail', '2', 'Elon Musk', '$180 B', '51', 'United States', 'Tesla, SpaceX', 'Automotive', '3', 'Jeff Bezos', '$114 B', '59', 'United States', 'Amazon', 'Technology', '4', 'Larry Ellison', '$107 B', '78', 'United States', 'Oracle', 'Technology', '5', 'Warren Buffett', '$106 B', '92', 'United States', 'Berkshire Hathaway', 'Finance & Investments', '6', 'Bill Gates', '$104 B', '67', 'United States', 'Microsoft', 'Technology', '7', 'Michael Bloomberg', '$94.5 B', '81', 'United States', 'Bloomberg LP', 'Media & Entertainment', '8', 'Carlos Slim Helu & family', '$93 B', '83', 'Mexico', 'Telecom', 'Telecom', '9', 'Mukesh Ambani', '$83.4 B', '65', 'India', 'Diversified', 'Diversified', '10', 'Steve Ballmer', '$80.7 B', '67', 'United States', 'Microsoft', 'Technology', '11', 'Francoise Bettencourt Meyers & family', '$80.5 B', '69', 'France', "L'Oréal", 'Fashion & Retail', '12', 'Larry Page', '$79.2 B', '50', 'United

In [44]:
# To scarpe data (i.e. “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”)

rank = []
name = []
net_worth = []
age = []
citizenship = []
source = []
industry = []

for i in range(0, len(all), 7):
    rank.append(all[i])
print(rank)

for i in range(1, len(all), 7):
    name.append(all[i])
print(name)

for i in range(2, len(all), 7):
    net_worth.append(all[i])
print(net_worth)

for i in range(3, len(all), 7):
    age.append(all[i])
print(age)

for i in range(4, len(all), 7):
    citizenship.append(all[i])
print(citizenship)

for i in range(5, len(all), 7):
    source.append(all[i])
print(source)

for i in range(6, len(all), 7):
    industry.append(all[i])
print(industry)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '17', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '31', '33', '34', '35', '35', '37', '38', '39', '40', '41', '41', '43', '43', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '65', '67', '68', '69', '70', '71', '72', '72', '74', '74', '76', '77', '77', '79', '80', '81', '82', '83', '84', '84', '86', '86', '88', '89', '89', '89', '92', '93', '94', '94', '94', '97', '97', '99', '100', '101', '101', '103', '104', '104', '106', '107', '108', '108', '108', '108', '112', '113', '114', '115', '116', '116', '118', '119', '120', '121', '122', '123', '124', '124', '126', '127', '128', '128', '130', '130', '130', '133', '133', '133', '136', '137', '138', '138', '140', '141', '142', '142', '144', '145', '146', '147', '148', '148', '148', '151', '151', '153', '153', '153', '153', '157', '157', '

In [45]:
button_next = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/button[2]/div")
button_next.click()

In [46]:
# To scarpe data (i.e. “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”)

a = []
b = []
c = []
d = []
e = []
f = []
g = []

while True:
    
    # All

    all = []
    all_element = driver.find_elements(By.XPATH, "//div[@class='TableRow_cell__db-hv Table_cell__houv9']")
    time.sleep(2)
        
    for i in all_element:
        all.append(i.text)
    
    for i in range(0, len(all), 7):
        a.append(all[i])
        #print(rank)
        
    for i in range(1, len(all), 7):
        b.append(all[i])
        #print(name)

    for i in range(2, len(all), 7):
        c.append(all[i])
        #print(net_worth)

    for i in range(3, len(all), 7):
        d.append(all[i])
        #print(age)

    for i in range(4, len(all), 7):
        e.append(all[i])
        #print(citizenship)

    for i in range(5, len(all), 7):
        f.append(all[i])
        #print(source)

    for i in range(6, len(all), 7):
        g.append(all[i])
        #print(industry)
    
    rank.extend(a)
    name.extend(b)
    net_worth.extend(c)
    age.extend(d)
    citizenship.extend(e)
    source.extend(f)
    industry.extend(g)
    
    time.sleep(3)
    try:
        button_next = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/button[2]/div")
        button_next.click()
    except:
        break
    time.sleep(4)

In [47]:
df = pd.DataFrame({'Rank': rank, 'Name' : name,  'Net Worth' : net_worth, 'Age' : age, 'Citizenship' : citizenship,  'Source' : source, 'Industry' : industry}) 
df

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
3,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
18235,2540,Yu Rong,$1 B,51,China,Health clinics,Healthcare
18236,2540,"Richard Yuengling, Jr.",$1 B,80,United States,Beer,Food & Beverage
18237,2540,Zhang Gongyun,$1 B,60,China,Tyre manufacturing machinery,Manufacturing
18238,2540,Zhang Guiping & family,$1 B,71,China,Real estate,Real Estate


In [48]:
df.to_csv('billionaires.csv', index=False)

## Q8 Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [292]:
driver = webdriver.Edge()

In [293]:
# open any youtube video

driver.get('https://www.youtube.com/watch?v=zHPaFDRZMUo')

In [300]:
# Scroll down to load more comments

for _ in range(90):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(2)  


In [301]:
# scrape comments
comments = []
a = []

cm = driver.find_elements(By.XPATH, '//div[@class="style-scope ytd-expander"]')

for i in cm:
    if i.text is '':
        a.append("--")
    else:
        a.append(i.text)
comments = a[2:]
time.sleep(4)

In [302]:
print(comments)
print(len(comments))

['Jesus....please forgive me for all my anger, hate, greed, addictions, and sins... Baptism me in your grace and mercy. Bless those who seek your face. I need you, Jesus. We need you, Jesus!', '"He will never change but You must change" - Amen this really touched me.', 'Beautiful message:-\n1) God has not changed\n2) The Word  of God (the Bible) has not changed\n3) The way to Salvation has not changed.', 'Jesus Christ is my lord and Savior. I got saved September 20th 2020', 'FATHER GOD THANK YOU FOR WAKING ME UP THIS MORNING AND WATCHING OVER ME AND MY FAMILY IN JESUS NAME AMEN', "I sleep in peace. Thank you and may you all find that peace that Jesus gives to us, the peace that surpasses man's understanding. ", 'This is a powerful and timely message—needed more than ever. The world around us is shifting and changing, but God and His word never change. Hallelujah!', 'Tq Lord for saving a sinner like me through the precious blood of Jesus Christ ', '“How can a finite mind like mine, comp

In [308]:
# scrape upvote
upvote = []
no_of_upvote = []

up = driver.find_elements(By.XPATH, '//span[@class="style-scope ytd-comment-action-buttons-renderer"]')

for i in up:
    if i.text is '':
        upvote.append('--')
    else:
        upvote.append(i.text)
    
        
for i in range(1,len(upvote),2):
    no_of_upvote.append(upvote[i])
    
time.sleep(4)


In [309]:
print(no_of_upvote)
print(len(no_of_upvote))

['750', '2.9K', '827', '37', '94', '34', '5', '57', '4', '374', '62', '2', '1.1K', '3', '4', '5', '4', '3', '4', '49', '23', '3', '12', '3', '866', '3', '9', '2', '8', '10', '298', '24', '2', '8', '3', '9', '531', '2', '3', '3', '3', '5', '22', '1', '2', '32', '7', '229', '3', '2', '95', '10', '4', '216', '2', '3', '2', '2', '2', '25', '2', '2', '2', '2', '1', '4', '2', '2', '3', '12', '2', '137', '2', '3', '7', '5', '5', '5', '4', '10', '4', '2', '12', '5', '1', '4', '7', '2', '5', '21', '3', '3', '2', '1', '14', '207', '--', '5', '4', '2', '5', '37', '1', '1', '2', '1', '5', '86', '1', '1', '1', '3', '5', '48', '1', '4', '2', '2', '2', '86', '4', '3', '6', '4', '2', '108', '1', '2', '2', '--', '1', '8', '1', '1', '1', '3', '3', '601', '4', '4', '1', '1', '3', '15', '1', '1', '1', '3', '1', '24', '1', '2', '1', '1', '1', '149', '1', '1', '1', '1', '1', '13', '2', '2', '1', '1', '4', '14', '3', '2', '3', '2', '1', '3', '2', '1', '1', '1', '1', '3', '1', '5', '2', '1', '5', '20', '1', '

In [306]:
# scrape time
import time
time1_comments = []

t = driver.find_elements(By.XPATH, '//a[contains(text(),"ago")]')

for i in t:
    if i.text is '':
        time1_comments.append('Not Present')
    else:
        time1_comments.append(i.text)

time.sleep(4)

In [307]:
print(time1_comments)
print(len(time1_comments))

['4 years ago', '4 years ago', '2 years ago', '1 year ago', '2 years ago', '9 months ago', '1 year ago', '1 year ago', '7 months ago', '3 years ago', '1 year ago', '3 weeks ago', '3 years ago', '1 month ago', '2 years ago', '1 year ago', '1 year ago', '1 year ago', '4 years ago', '2 years ago', '1 year ago', '1 month ago', '1 year ago', '2 years ago', '4 years ago', '1 year ago', '6 months ago', '4 months ago', '3 months ago', '2 years ago', '3 years ago', '2 years ago', '4 months ago', '9 months ago', '2 years ago', '2 years ago', '4 years ago', '2 months ago', '2 years ago', '2 years ago', '5 months ago', '2 years ago', '3 years ago', '2 months ago', '1 year ago', '2 years ago', '2 years ago', '4 years ago', '2 years ago', '5 months ago', '3 years ago', '2 years ago', '2 years ago', '4 years ago', '3 years ago', '1 year ago', '2 years ago', '1 year ago', '2 years ago', '4 years ago', '1 month ago', '6 months ago', '3 years ago', '2 years ago', '2 years ago', '3 years ago', '1 year ag

# Q8 Output

In [310]:
df = pd.DataFrame({
    'Comments' : comments[:600],
    'Comment Upvote':no_of_upvote[:600],
    'Time':time1_comments[:600],
})
df

,Comments,Comment Upvote,Time
0,"Jesus....please forgive me for all my anger, h...",750,4 years ago
1,"""He will never change but You must change"" - A...",2.9K,4 years ago
2,Beautiful message:-\n1) God has not changed\n2...,827,2 years ago
3,Jesus Christ is my lord and Savior. I got save...,37,1 year ago
4,FATHER GOD THANK YOU FOR WAKING ME UP THIS MOR...,94,2 years ago
...,...,...,...
595,Amen ️️,1,2 years ago
596,"1 Peter 4:8️\n\nAbove all, love each other, fo...",1,5 months ago
597,God please I have broken so many promises Lord...,--,2 years ago
598,AMEN AMEN AMEN,1,1 year ago


In [311]:
df.to_csv('Youtube_Comments.csv', index=False)

## Q9 Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. 
You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, 
dorms from price, facilities and property description. 

In [143]:
driver = webdriver.Edge()

In [144]:
# Go to webpage https://www.hostelworld.com/

driver.get('https://www.hostelworld.com/')

In [145]:
# search for the location "London"

location_search = driver.find_element(By.XPATH, "//input[@class='native-input']")  
location_search.send_keys("London")

time.sleep(3)  
   

In [146]:
# click on the search icon

select_location = driver.find_element(By.XPATH, "//button[@class='item-content']")  
select_location.click()

In [147]:
# click on the search icon

button_location = driver.find_element(By.XPATH, "//button[@class='btn-content medium-button icon-only']")  
button_location.click()
time.sleep(4)

In [148]:
# You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, 
# privates from price, dorms from price, facilities and property description.

# 1. hostel name
try:
    hostel_name = []
    a = []
    
    hostel_name_element = driver.find_elements(By.XPATH, "//div[@class='property-name']")
    time.sleep(2)
        
    for i in hostel_name_element:
            a.append(i.text)
    hostel_name = a[2:]
    
except NoSuchElementException:
    hostel_name.append('-')

In [149]:
print(hostel_name)
print(len(hostel_name))

["St Christopher's Village", 'Onefam Notting Hill', 'Urbany Hostel London', 'Generator London', 'NX London Hostel', 'Safestay London Elephant & Castle', 'Palmers Lodge - Swiss Cottage', 'Safestay London Kensington Holland Park', 'Clink261', 'No.8 Willesden Hostel London', 'Destinations Hostels @ The Gallery', 'Phoenix Hostel', 'Queen Elizabeth Chelsea', 'No.8 Seven Sisters', 'London Backpackers', 'Hostelle - women only hostel London', 'Prime Backpackers Angel', "St Christopher's Hammersmith", 'Kabannas London St Pancras', "St Christopher's Camden", "St Christopher's Inn - London Bridge", "St Christopher's Greenwich", 'Saint James Backpackers', 'Bell House Hostel', "St Christopher's Shepherds Bush", 'PubLove @ The Steam Engine, Waterloo', 'Book a Bed Hostels', 'The Backpackshack', "St Christopher's Oasis-Female Only", "St Christopher's Inn - Liverpool Street"]
30


In [150]:
# You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, 
# privates from price, dorms from price, facilities and property description.

# 2. distance from city centre
try:
    distance = []
    distance_element = driver.find_elements(By.XPATH, "//span[@class='distance-description']")
    time.sleep(2)
        
    for i in distance_element:
        distance.append(i.text)
except NoSuchElementException:
    distance.append('-')

In [151]:
print(distance)
print(len(distance))

['- 1.8km from city centre', '- 5.5km from city centre', '- 5.4km from city centre', '- 3km from city centre', '- 6.1km from city centre', '- 1.7km from city centre', '- 6.5km from city centre', '- 5.8km from city centre', '- 3.2km from city centre', '- 10km from city centre', '- 1.7km from city centre', '- 4.2km from city centre', '- 5.7km from city centre', '- 9km from city centre', '- 11.9km from city centre', '- 5.1km from city centre', '- 3.6km from city centre', '- 7.5km from city centre', '- 3.3km from city centre', '- 4.3km from city centre', '- 1.8km from city centre', '- 7.6km from city centre', '- 5.5km from city centre', '- 4.2km from city centre', '- 7km from city centre', '- 0.5km from city centre', '- 6.9km from city centre', '- 10.9km from city centre', '- 1.8km from city centre', '- 3.2km from city centre']
30


In [152]:
# You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, 
# privates from price, dorms from price, facilities and property description.

# 3. ratings
try:
    ratings = []
    a = []
    ratings_element = driver.find_elements(By.XPATH, "//span[@class='number']")
    time.sleep(2)
     
    for i in ratings_element:
        a.append(i.text)
    ratings = a[2:]
    
except NoSuchElementException:
    ratings.append('-')

In [153]:
print(ratings)
print(len(ratings))

['8.2', '9.7', '9.5', '7.6', '8.0', '7.0', '9.0', '7.0', '8.0', '6.9', '9.2', '7.3', '7.7', '5.6', '8.0', '9.0', '8.0', '7.5', '8.3', '7.5', '7.7', '7.5', '7.5', '7.6', '7.0', '7.8', '7.5', '8.3', '8.1', '7.7']
30


In [154]:
# You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, 
# privates from price, dorms from price, facilities and property description.

# 4. total reviews

try:
    total_reviews = []
    total_reviews_element = driver.find_elements(By.XPATH, "//span[@class='left-margin']")
    time.sleep(2)
        
    for i in total_reviews_element:
        total_reviews.append(i.text)
except NoSuchElementException:
    total_reviews.append('-')

In [155]:
print(total_reviews)
print(len(total_reviews))

['(12484)', '(2296)', '(941)', '(7802)', '(2002)', '(5133)', '(15716)', '(1668)', '(420)', '(5100)', '(240)', '(4455)', '(3585)', '(4071)', '(4526)', '(50)', '(808)', '(4307)', '(2913)', '(4096)', '(3551)', '(3399)', '(1924)', '(99)', '(800)', '(430)', '(1278)', '(780)', '(773)', '(532)']
30


In [156]:
# You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, 
# privates from price, dorms from price, facilities and property description.

# 5. overall reviews
try:
    overall_reviews = []
    a = []
    overall_reviews_element = driver.find_elements(By.XPATH, "//span[@class='keyword']")
    time.sleep(2)
        
    for i in overall_reviews_element:
        a.append(i.text)
    overall_reviews = a[2:]
        
except NoSuchElementException:
    overall_reviews.append('-')

In [161]:
print(overall_reviews)
print(len(overall_reviews))

['Fabulous', 'Superb', 'Superb', 'Very Good', 'Fabulous', 'Very Good', 'Superb', 'Very Good', 'Fabulous', 'Good', 'Superb', 'Very Good', 'Very Good', 'Fabulous', 'Superb', 'Fabulous', 'Very Good', 'Fabulous', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Fabulous', 'Very Good']
29


In [162]:
# You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, 
# privates from price, dorms from price, facilities and property description.

# 5. privates from price and dorms from price
try:
    private_dorm = []
    private_dorm_element = driver.find_elements(By.XPATH, "//div[@class='property-accommodation-prices']")
    time.sleep(2)
        
    for i in private_dorm_element:
        private_dorm.append(i.text)
        
except NoSuchElementException:
    private_dorm.append('-')

In [163]:
print(private_dorm)
print(len(private_dorm))

['No Privates Available\n-10%\nDorms From\n₹3,034\n₹2,730', 'Privates From\n₹18,721\nNo Dorms Available', 'Privates From\n₹14,497\nDorms From\n₹2,927', 'Privates From\n₹10,424\nDorms From\n₹2,530', 'Privates From\n₹11,583\nDorms From\n₹2,741', 'Privates From\n₹13,421\nDorms From\n₹1,883', 'No Privates Available\n-26%\nDorms From\n₹3,759\n₹2,782', 'No Privates Available\nDorms From\n₹1,744', 'No Privates Available\nDorms From\n₹2,611', 'Privates From\n₹2,637\nDorms From\n₹1,214', 'No Privates Available\nDorms From\n₹5,334', 'No Privates Available\n-5%\nDorms From\n₹1,974\n₹1,875', 'No Privates Available\nDorms From\n₹1,883', 'No Privates Available\nDorms From\n₹1,316', 'Privates From\n₹7,112\nDorms From\n₹1,829', 'No Privates Available\nDorms From\n₹3,658', 'No Privates Available\nDorms From\n₹2,681', 'No Privates Available\n-10%\nDorms From\n₹2,395\n₹2,155', 'No Privates Available\nDorms From\n₹4,802', 'No Privates Available\n-10%\nDorms From\n₹1,937\n₹1,743', 'No Privates Available\n-

In [164]:
#fetching url of each hostel
hostel_url = []

url = driver.find_elements(By.XPATH, "//a[@class='property-card-container horizontal']")
for i in url:
    hostel_url.append(i.get_attribute("href"))

print(hostel_url)
print(len(hostel_url))

["https://www.hostelworld.com/pwa/hosteldetails.php/St%20Christopher's%20Village/London/502?from=2023-10-02&to=2023-10-05&guests=2", 'https://www.hostelworld.com/pwa/hosteldetails.php/Onefam%20Notting%20Hill/London/291433?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Urbany%20Hostel%20London/London/303427?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Generator%20London/London/510?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/NX%20London%20Hostel/London/15204?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Safestay%20London%20Elephant%20&%20Castle/London/55551?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Palmers%20Lodge%20-%20Swiss%20Cottage/London/14348?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Safestay%20London

In [166]:
# do click on more button for product description and facilities

button = driver.find_element(By.XPATH,'//button[@class = "pill-content page-nav nav-right icon-only"]')
button.click()
time.sleep(8)

In [167]:
# page 2

# 1. hostel name
try:
    a = []
    
    hostel_name_element = driver.find_elements(By.XPATH, "//div[@class='property-name']")
    time.sleep(2)
        
    for i in hostel_name_element:
            a.append(i.text)
            
    hostel_name.extend(a)
    
except NoSuchElementException:
    hostel_name.append('-')

In [168]:
print(hostel_name)
print(len(hostel_name))

["St Christopher's Village", 'Onefam Notting Hill', 'Urbany Hostel London', 'Generator London', 'NX London Hostel', 'Safestay London Elephant & Castle', 'Palmers Lodge - Swiss Cottage', 'Safestay London Kensington Holland Park', 'Clink261', 'No.8 Willesden Hostel London', 'Destinations Hostels @ The Gallery', 'Phoenix Hostel', 'Queen Elizabeth Chelsea', 'No.8 Seven Sisters', 'London Backpackers', 'Hostelle - women only hostel London', 'Prime Backpackers Angel', "St Christopher's Hammersmith", 'Kabannas London St Pancras', "St Christopher's Camden", "St Christopher's Inn - London Bridge", "St Christopher's Greenwich", 'Saint James Backpackers', 'Bell House Hostel', "St Christopher's Shepherds Bush", 'PubLove @ The Steam Engine, Waterloo', 'Book a Bed Hostels', 'The Backpackshack', "St Christopher's Oasis-Female Only", "St Christopher's Inn - Liverpool Street", 'Selina Camden', 'PubLove @ The Crown, Battersea', 'Park Villa', 'PubLove @ The White Ferry, Victoria', 'PubLove @ The Rose & Cr

In [169]:
# Page 2

# 2. distance from city centre
try:
    a = []
    distance_element = driver.find_elements(By.XPATH, "//span[@class='distance-description']")
    time.sleep(2)
        
    for i in distance_element:
        a.append(i.text)
    
    distance.extend(a)
        
except NoSuchElementException:
    distance.append('-')

In [170]:
print(distance)
print(len(distance))

['- 1.8km from city centre', '- 5.5km from city centre', '- 5.4km from city centre', '- 3km from city centre', '- 6.1km from city centre', '- 1.7km from city centre', '- 6.5km from city centre', '- 5.8km from city centre', '- 3.2km from city centre', '- 10km from city centre', '- 1.7km from city centre', '- 4.2km from city centre', '- 5.7km from city centre', '- 9km from city centre', '- 11.9km from city centre', '- 5.1km from city centre', '- 3.6km from city centre', '- 7.5km from city centre', '- 3.3km from city centre', '- 4.3km from city centre', '- 1.8km from city centre', '- 7.6km from city centre', '- 5.5km from city centre', '- 4.2km from city centre', '- 7km from city centre', '- 0.5km from city centre', '- 6.9km from city centre', '- 10.9km from city centre', '- 1.8km from city centre', '- 3.2km from city centre', '- 5.5km from city centre', '- 4.7km from city centre', '- 6.3km from city centre', '- 2.4km from city centre', '- 1.6km from city centre', '- 0.7km from city centr

In [171]:
# Page 2

# 3. ratings
try:
    a = []
    ratings_element = driver.find_elements(By.XPATH, "//span[@class='number']")
    time.sleep(2)
     
    for i in ratings_element:
        a.append(i.text)
        
    ratings.extend(a)
    
except NoSuchElementException:
    ratings.append('-')

In [173]:
print(ratings)
print(len(ratings))

['8.2', '9.7', '9.5', '7.6', '8.0', '7.0', '9.0', '7.0', '8.0', '6.9', '9.2', '7.3', '7.7', '5.6', '8.0', '9.0', '8.0', '7.5', '8.3', '7.5', '7.7', '7.5', '7.5', '7.6', '7.0', '7.8', '7.5', '8.3', '8.1', '7.7', '8.8', '7.7', '8.5', '6.9', '8.2', '6.1', '8.3', '5.2', '7.4', '6.7', '6.9', '5.5', '6.6', '10', '10', '4.9', '7.7', '6.6', '4.7']
49


In [174]:
# Page 2

# 4. total reviews

try:
    a = []
    total_reviews_element = driver.find_elements(By.XPATH, "//span[@class='left-margin']")
    time.sleep(2)
        
    for i in total_reviews_element:
        a.append(i.text)
    
    total_reviews.extend(a)
    
except NoSuchElementException:
    total_reviews.append('-')

In [175]:
print(total_reviews)
print(len(total_reviews))

['(12484)', '(2296)', '(941)', '(7802)', '(2002)', '(5133)', '(15716)', '(1668)', '(420)', '(5100)', '(240)', '(4455)', '(3585)', '(4071)', '(4526)', '(50)', '(808)', '(4307)', '(2913)', '(4096)', '(3551)', '(3399)', '(1924)', '(99)', '(800)', '(430)', '(1278)', '(780)', '(773)', '(532)', '(55)', '(312)', '(918)', '(379)', '(206)', '(2550)', '(1524)', '(3730)', '(176)', '(1090)', '(42)', '(452)', '(428)', '(151)', '(1383)', '(1)', '(55)', '(30)', '(366)']
49


In [176]:
# page 2

# 5. overall reviews
try:
    a = []
    overall_reviews_element = driver.find_elements(By.XPATH, "//span[@class='keyword']")
    time.sleep(2)
        
    for i in overall_reviews_element:
        a.append(i.text)
        
    overall_reviews.extend(a)
        
except NoSuchElementException:
    overall_reviews.append('-')

In [177]:
print(overall_reviews)
print(len(overall_reviews))

['Fabulous', 'Superb', 'Superb', 'Very Good', 'Fabulous', 'Very Good', 'Superb', 'Very Good', 'Fabulous', 'Good', 'Superb', 'Very Good', 'Very Good', 'Fabulous', 'Superb', 'Fabulous', 'Very Good', 'Fabulous', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Fabulous', 'Very Good', 'Fabulous', 'Very Good', 'Fabulous', 'Good', 'Fabulous', 'Good', 'Fabulous', 'Very Good', 'Good', 'Good', 'Good', 'Superb', 'Superb', 'Very Good', 'Good']
44


In [178]:
# page 2

# 5. privates from price and dorms from price
try:
    a = []
    private_dorm_element = driver.find_elements(By.XPATH, "//div[@class='property-accommodation-prices']")
    time.sleep(2)
        
    for i in private_dorm_element:
        a.append(i.text)
    
    private_dorm.extend(a)
        
except NoSuchElementException:
    private_dorm.append('-')

In [179]:
print(private_dorm)
print(len(private_dorm))

['No Privates Available\n-10%\nDorms From\n₹3,034\n₹2,730', 'Privates From\n₹18,721\nNo Dorms Available', 'Privates From\n₹14,497\nDorms From\n₹2,927', 'Privates From\n₹10,424\nDorms From\n₹2,530', 'Privates From\n₹11,583\nDorms From\n₹2,741', 'Privates From\n₹13,421\nDorms From\n₹1,883', 'No Privates Available\n-26%\nDorms From\n₹3,759\n₹2,782', 'No Privates Available\nDorms From\n₹1,744', 'No Privates Available\nDorms From\n₹2,611', 'Privates From\n₹2,637\nDorms From\n₹1,214', 'No Privates Available\nDorms From\n₹5,334', 'No Privates Available\n-5%\nDorms From\n₹1,974\n₹1,875', 'No Privates Available\nDorms From\n₹1,883', 'No Privates Available\nDorms From\n₹1,316', 'Privates From\n₹7,112\nDorms From\n₹1,829', 'No Privates Available\nDorms From\n₹3,658', 'No Privates Available\nDorms From\n₹2,681', 'No Privates Available\n-10%\nDorms From\n₹2,395\n₹2,155', 'No Privates Available\nDorms From\n₹4,802', 'No Privates Available\n-10%\nDorms From\n₹1,937\n₹1,743', 'No Privates Available\n-

In [180]:
#fetching url of each hostel - page 2

hostel_url1 = []

url1 = driver.find_elements(By.XPATH, "//a[@class='property-card-container horizontal']")
for i in url1:
    hostel_url1.append(i.get_attribute("href"))

print(hostel_url1)
print(len(hostel_url1))

['https://www.hostelworld.com/pwa/hosteldetails.php/Selina%20Camden/London/310274?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove%20@%20The%20Crown,%20Battersea/London/60336?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Park%20Villa/London/98066?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove%20@%20The%20White%20Ferry,%20Victoria/London/39761?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove%20@%20The%20Rose%20&%20Crown/London/297123?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/London%20Waterloo%20Hostel/London/39493?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Hootananny%20Hostel/London/24068?from=2023-10-02&to=2023-10-05&guests=2', 'https://www.hostelworld.com/pwa/hosteldetails.php/Kensal%20Gree

In [181]:
 # facilities and property description.

property1 = []

for i in hostel_url:
    driver.get(i)
    time.sleep(3)
    
    try:
        property_element = driver.find_element(By.XPATH, '//div[@class="content"]')
        property1.append(property_element.text)
        
    except NoSuchElementException:
        property1.append('Not Present')
    
print(len(property1))
        

30


In [182]:
print(property1)

['165 Borough High Street, London, England', "63 Prince's Square, London, England", '48-49 Princes Square, Paddington, London, England', 'Compton Place, Off 37 Tavistock Place, WC1, London, England', '323 New Cross Road, London, England', '144-152 Walworth Road, London, England', '40 College Crescent, Swiss Cottage, London, England', 'Holland Park Ave, Holland Walk, London, England', '261-265 Grays Inn Road, Kings Cross, London, England', '305 High Rd, Willesden, London, England', '1 Lupus street, Pimlico, London, England', '6 Daventry Street, London, England', '58 Bagleys Lane, London, England', '618 Seven Sisters Rd, Seven Sisters, London, England', '8/10 Queens Parade, Queens Road, Hendon Central, London, England', '458 Bethnal Green road, London, England', '333 City Road, 333 City Road, London, England', '28, Hammersmith Broadway, Hammersmith, London, England', '79-81 Euston Road, London, England', '48-50 Camden High Street, Camden NW1, London, England', '121 Borough High Street, L

In [183]:
  
# facilities1

facilities1 = []

for i in hostel_url:
    driver.get(i)
    time.sleep(3)
    
    try:
        facilities1_element = driver.find_element(By.XPATH, '//ul[@class="facilities"]')
        facilities1.append(facilities1_element.text)
        
    except NoSuchElementException:
        facilities1.append('Not Present')
    
print(len(facilities1))
print(facilities1) 



30
['Linen Included Free WiFi Free Internet Access', 'Linen Included Free WiFi Free Internet Access', 'Linen Included Free WiFi', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Free Breakfast Linen Included Towels Included Free WiFi', 'Linen Included Free WiFi', 'Linen Included Free WiFi Free Internet Access', 'Linen Included Free WiFi', 'Linen Included Free City Maps Free WiFi', 'Linen Included Free WiFi', 'Free Breakfast Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free WiFi', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Towels Included Free WiFi Free Internet Access Free City Tour', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi', 'Linen

In [184]:
 # page 2

a = []

for i in hostel_url1:
    driver.get(i)
    time.sleep(3)
    
    try:
        property_element = driver.find_element(By.XPATH, '//div[@class="content"]')
        a.append(property_element.text)
        
    except NoSuchElementException:
        a.append('Not Present')
    
property1.extend(a)
print(len(property1))

50


In [185]:
print(property1)
print(len(property1))

['165 Borough High Street, London, England', "63 Prince's Square, London, England", '48-49 Princes Square, Paddington, London, England', 'Compton Place, Off 37 Tavistock Place, WC1, London, England', '323 New Cross Road, London, England', '144-152 Walworth Road, London, England', '40 College Crescent, Swiss Cottage, London, England', 'Holland Park Ave, Holland Walk, London, England', '261-265 Grays Inn Road, Kings Cross, London, England', '305 High Rd, Willesden, London, England', '1 Lupus street, Pimlico, London, England', '6 Daventry Street, London, England', '58 Bagleys Lane, London, England', '618 Seven Sisters Rd, Seven Sisters, London, England', '8/10 Queens Parade, Queens Road, Hendon Central, London, England', '458 Bethnal Green road, London, England', '333 City Road, 333 City Road, London, England', '28, Hammersmith Broadway, Hammersmith, London, England', '79-81 Euston Road, London, England', '48-50 Camden High Street, Camden NW1, London, England', '121 Borough High Street, L

In [186]:
# Page 2

# facilities1

a = []

for i in hostel_url1:
    driver.get(i)
    time.sleep(3)
    
    try:
        facilities1_element = driver.find_element(By.XPATH, '//ul[@class="facilities"]')
        a.append(facilities1_element.text)
        
    except NoSuchElementException:
        a.append('Not Present')

facilities1.extend(a) 

In [187]:
print(len(facilities1))
print(facilities1) 

50
['Linen Included Free WiFi Free Internet Access', 'Linen Included Free WiFi Free Internet Access', 'Linen Included Free WiFi', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Free Breakfast Linen Included Towels Included Free WiFi', 'Linen Included Free WiFi', 'Linen Included Free WiFi Free Internet Access', 'Linen Included Free WiFi', 'Linen Included Free City Maps Free WiFi', 'Linen Included Free WiFi', 'Free Breakfast Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free WiFi', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Towels Included Free WiFi Free Internet Access Free City Tour', 'Linen Included Free City Maps Free WiFi Free Internet Access', 'Linen Included Free City Maps Free WiFi', 'Linen

In [190]:
print(len(hostel_name), len(distance), len(ratings), len(total_reviews), len(overall_reviews), len(private_dorm), len(property1), len(facilities1))

50 50 49 49 44 50 50 50


## Q9 Output

In [192]:
df = pd.DataFrame({
    'Hostel Name' : hostel_name[:44],
    'Distance':distance[:44],
    'Ratings':ratings[:44],
    'Total Reviews':total_reviews[:44],
    'Overall Reviews':overall_reviews[:44],
    'Private Dorm': private_dorm[:44],
    'Property': property1[:44],
    'Facilities': facilities1[:44]
})
df


,Hostel Name,Distance,Ratings,Total Reviews,Overall Reviews,Private Dorm,Property,Facilities
0,St Christopher's Village,- 1.8km from city centre,8.2,(12484),Fabulous,"No Privates Available\n-10%\nDorms From\n₹3,03...","165 Borough High Street, London, England",Linen Included Free WiFi Free Internet Access
1,Onefam Notting Hill,- 5.5km from city centre,9.7,(2296),Superb,"Privates From\n₹18,721\nNo Dorms Available","63 Prince's Square, London, England",Linen Included Free WiFi Free Internet Access
2,Urbany Hostel London,- 5.4km from city centre,9.5,(941),Superb,"Privates From\n₹14,497\nDorms From\n₹2,927","48-49 Princes Square, Paddington, London, England",Linen Included Free WiFi
3,Generator London,- 3km from city centre,7.6,(7802),Very Good,"Privates From\n₹10,424\nDorms From\n₹2,530","Compton Place, Off 37 Tavistock Place, WC1, Lo...",Linen Included Free City Maps Free WiFi Free I...
4,NX London Hostel,- 6.1km from city centre,8.0,(2002),Fabulous,"Privates From\n₹11,583\nDorms From\n₹2,741","323 New Cross Road, London, England",Free Breakfast Linen Included Towels Included ...
5,Safestay London Elephant & Castle,- 1.7km from city centre,7.0,(5133),Very Good,"Privates From\n₹13,421\nDorms From\n₹1,883","144-152 Walworth Road, London, England",Linen Included Free WiFi
6,Palmers Lodge - Swiss Cottage,- 6.5km from city centre,9.0,(15716),Superb,"No Privates Available\n-26%\nDorms From\n₹3,75...","40 College Crescent, Swiss Cottage, London, En...",Linen Included Free WiFi Free Internet Access
7,Safestay London Kensington Holland Park,- 5.8km from city centre,7.0,(1668),Very Good,"No Privates Available\nDorms From\n₹1,744","Holland Park Ave, Holland Walk, London, England",Linen Included Free WiFi
8,Clink261,- 3.2km from city centre,8.0,(420),Fabulous,"No Privates Available\nDorms From\n₹2,611","261-265 Grays Inn Road, Kings Cross, London, E...",Linen Included Free City Maps Free WiFi
9,No.8 Willesden Hostel London,- 10km from city centre,6.9,(5100),Good,"Privates From\n₹2,637\nDorms From\n₹1,214","305 High Rd, Willesden, London, England",Linen Included Free WiFi


In [193]:
df.to_csv('London_Hostels.csv', index=False)